In [233]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, basinhopping, differential_evolution

# SimpleWorld Data

In [5]:
# Create the SimpleWorld individual data

ids = range(1,6)
ages = [59,54,35,73,49]
sex = ["m","m","m","f","f"]
income = [2868, 2474, 2231, 3152, 2473]

ind = pd.DataFrame(np.transpose([ids,ages,sex,income]), columns=["id","age","sex","income"])

# Drop the income field
ind = ind.drop(['income'],axis=1)

# Now recategorise the age variable
ind['age'] = pd.to_numeric(ind['age'])

# overwrite the age variable with categorical age
ind['age'] = pd.cut(ind['age'], [0,49,120], labels = ['a0_49','a50+'])

# As the dimensions are different, we need to 'flatten' the individual dataset.
# This means that responses become fields, and values become booleans, with rows reflecting individuals
age_pivot = pd.pivot_table(ind,columns=['age'],values='id', index=ind.index, aggfunc=len, fill_value=0)
# The last square bracket bit ensures that the column order is male then female.
sex_pivot = pd.pivot_table(ind,columns=['sex'],values='id', index=ind.index, aggfunc=len, fill_value=0)[['m','f']]

# merge pivoted data to make flat dataframe
ind_cat = pd.DataFrame(age_pivot.to_records()).merge(pd.DataFrame(sex_pivot.to_records()),left_index=True,right_index=True)
# drop nuisance columns
ind_cat = ind_cat.drop(['index_x','index_y'],axis=1)
ind_cat

,a0_49,a50+,m,f
0,0,1,1,0
1,0,1,1,0
2,1,0,1,0
3,0,1,0,1
4,1,0,0,1


In [6]:
# Create the SimpleWorld constraints data

# SimpleWorld age constraint data
con_age = pd.DataFrame([[8,4],[2,8],[7,4]],columns=["0-49","50+"])
# SimpleWorld sex constraint data
con_sex = pd.DataFrame([[6,6],[4,6],[3,8]],columns=["m","f"])
# Rename the con_age fields to match the categories in the ind table
con_age = con_age.rename(columns={'0-49':'a0_49','50+':'a50+'})
# Finally create a single contraint object by merging the constraints tables.
cons = con_age.merge(con_sex,left_index=True,right_index=True)
cons

,a0_49,a50+,m,f
0,8,4,6,6
1,2,8,4,6
2,7,4,3,8


# Optimisation Specific Data Setup

In [96]:
# Records in the individual table need to be unique.
# Individuals who are duplicates on the dimensions used in the spatial microsimulation must be removed.
# Then a frequency column should be added to preserve duplicated rows.

def unique_mat_freqs(ind):
    patterns = ind.copy()
    # Get the constraint 'pattern'
    cols = patterns.columns
    patterns['pattern'] = patterns.apply(lambda x: ''.join(str(e) for e in[x[c] for c in cols]),axis=1)
    # Get the frequencies of each pattern
    freq = patterns.groupby('pattern').size()
    # Get indices of unique rows
    ind_unique_index = patterns.drop_duplicates().index
    # Select unique patterns
    patterns = patterns.iloc[ind_unique_index]
    # merge back freq
    patterns = patterns.merge(freq.to_frame(name='freq'),left_on='pattern',right_index=True)
    # Finally store original row indices and reindex
    patterns['rns'] = patterns.index
    patterns = patterns.reset_index(drop=True)
    return patterns

# Call this function to return an individuals table with no duplicates
umat = unique_mat_freqs(ind_cat)

# Now remake the binary individuals matrix to reflect counts
indu = umat.iloc[:,range(0,len(ind_cat)-1)] * np.repeat(umat['freq'].values,len(umat)).reshape((len(umat),-1))

# We can now see that there are 2 50+ males in our dataset
indu

,a0_49,a50+,m,f
0,0,2,2,0
1,1,0,1,0
2,0,1,0,1
3,1,0,0,1


# Optimisation Methods for Reweighting

In [147]:
# Using the minimize function we need to set up a few things
# fun = objective function; here we'll use the total absolute error (TAE) per category
# x0 = initial guess at parameters, we'll use a vector of 1s.
# args = extra arguements to pass to the objective function. We need to pass the constraints, and frequency of individuals.
# method = type of solver, initially we'll use the conjugate gradient 'CG' solver.

def tae(x0, ind_freq, cons):
    # total absolute error compares the simulation to the real constraints
    ae = abs((ind_freq.apply(lambda x: x * x0[x.index], axis=0).sum(axis=0)) - cons)
    return ae.sum()

In [150]:
# Check that the objective function works with some suggested x0s for area 1
tae(np.array([1.2,3.5,1.5,4.5]),indu,cons.iloc[0])

0.19999999999999973

## Nelder-Mead

In [182]:
nm = minimize(fun=tae, x0 = np.ones(len(indu)), args = (indu,cons.iloc[0]), method = 'Nelder-Mead', tol=1e-6)
nm

 final_simplex: (array([[ 1.26561641,  3.46876723,  1.46876712,  4.53123261],
       [ 1.26561614,  3.46876776,  1.46876763,  4.53123206],
       [ 1.26561624,  3.46876765,  1.46876734,  4.5312326 ],
       [ 1.26561616,  3.46876778,  1.46876777,  4.53123251],
       [ 1.26561633,  3.46876678,  1.46876714,  4.53123307]]), array([  5.38388953e-07,   6.24531496e-07,   6.31697931e-07,
         7.56104382e-07,   1.11254030e-06]))
           fun: 5.3838895253122132e-07
       message: 'Optimization terminated successfully.'
          nfev: 273
           nit: 152
        status: 0
       success: True
             x: array([ 1.26561641,  3.46876723,  1.46876712,  4.53123261])

In [179]:
# Get weights for original ind table by duplicating weights where required
weights = np.repeat(nm.x,umat['freq'])
weights

array([ 1.26561641,  1.26561641,  3.46876723,  1.46876712,  4.53123261])

In [180]:
# Check that the weights make sense
print "Optimisation Weights"
print ind_cat.apply(lambda x: x*weights,axis=0).sum(axis=0)
print ""
print "Original Constraints"
print cons.iloc[0]

Optimisation Weights
a0_49    8.0
a50+     4.0
m        6.0
f        6.0
dtype: float64

Original Constraints
a0_49    8
a50+     4
m        6
f        6
Name: 0, dtype: int64


## Sequential Least SQuares Programming

In [188]:
# Now it's easy to swap out the different optimisers
slsqp = minimize(fun=tae, x0 = np.ones(len(indu)), args = (indu,cons.iloc[0]), method = 'SLSQP', tol=1e-7)
slsqp

     fun: 1.791446377907846e-07
     jac: array([-4.,  0., -2.,  0.])
 message: 'Optimization terminated successfully.'
    nfev: 254
     nit: 36
    njev: 36
  status: 0
 success: True
       x: array([ 1.15384618,  3.69230772,  1.69230765,  4.30769227])

In [189]:
# Get weights for original ind table by duplicating weights where required
weights = np.repeat(slsqp.x,umat['freq'])
weights

array([ 1.15384618,  1.15384618,  3.69230772,  1.69230765,  4.30769227])

In [190]:
# Check that the weights make sense
print "Optimisation Weights"
print ind_cat.apply(lambda x: x*weights,axis=0).sum(axis=0)
print ""
print "Original Constraints"
print cons.iloc[0]

Optimisation Weights
a0_49    8.0
a50+     4.0
m        6.0
f        6.0
dtype: float64

Original Constraints
a0_49    8
a50+     4
m        6
f        6
Name: 0, dtype: int64


## Constrained Optimization BY Linear Approximation

In [199]:
cobyla = minimize(fun=tae, x0 = np.ones(len(indu)), args = (indu,cons.iloc[0]), method = 'COBYLA', tol=1e-7)

In [200]:
# Get weights for original ind table by duplicating weights where required
weights = np.repeat(cobyla.x,umat['freq'])
weights

array([ 1.14687573,  1.14687573,  3.70624832,  1.70624853,  4.29375162])

In [201]:
# Check that the weights make sense
print "Optimisation Weights"
print ind_cat.apply(lambda x: x*weights,axis=0).sum(axis=0)
print ""
print "Original Constraints"
print cons.iloc[0]

Optimisation Weights
a0_49    8.0
a50+     4.0
m        6.0
f        6.0
dtype: float64

Original Constraints
a0_49    8
a50+     4
m        6
f        6
Name: 0, dtype: int64


## Basinhopping

In [229]:
# This algorithm is stochastic, so will likely produce different results run to run.
basin = basinhopping(func=tae, x0 = np.ones(len(indu)), niter=100, stepsize=0.1,
                     minimizer_kwargs = {'method':'SLSQP','args':(indu,cons.iloc[0]),'tol':1e-6})
basin

                        fun: 5.244393523895496e-07
 lowest_optimization_result:      fun: 5.244393523895496e-07
     jac: array([-4., -2., -2., -2.])
 message: 'Optimization terminated successfully.'
    nfev: 140
     nit: 18
    njev: 18
  status: 0
 success: True
       x: array([ 1.08792612,  3.82414768,  1.82414749,  4.17585249])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 16007
                        nit: 100
                       njev: 2076
                          x: array([ 1.08792612,  3.82414768,  1.82414749,  4.17585249])

In [230]:
# Get weights for original ind table by duplicating weights where required
weights = np.repeat(basin.x,umat['freq'])
weights

array([ 1.08792612,  1.08792612,  3.82414768,  1.82414749,  4.17585249])

In [231]:
# Check that the weights make sense
print "Optimisation Weights"
print ind_cat.apply(lambda x: x*weights,axis=0).sum(axis=0)
print ""
print "Original Constraints"
print cons.iloc[0]

Optimisation Weights
a0_49    8.0
a50+     4.0
m        6.0
f        6.0
dtype: float64

Original Constraints
a0_49    8
a50+     4
m        6
f        6
Name: 0, dtype: int64


## Differential Evolution

In [239]:
diff_ev = differential_evolution(func=tae, bounds = [(0,10),(0,10),(0,10),(0,10)], args = (indu,cons.iloc[0]))

In [240]:
# Get weights for original ind table by duplicating weights where required
weights = np.repeat(diff_ev.x,umat['freq'])
weights

array([ 0.93216183,  0.93216183,  4.13567633,  2.13567633,  3.86432367])

In [242]:
# Check that the weights make sense
print "Optimisation Weights"
print ind_cat.apply(lambda x: x*weights,axis=0).sum(axis=0)
print ""
print "Original Constraints"
print cons.iloc[0]

Optimisation Weights
a0_49    8.0
a50+     4.0
m        6.0
f        6.0
dtype: float64

Original Constraints
a0_49    8
a50+     4
m        6
f        6
Name: 0, dtype: int64


In [243]:
# All of these algorithms produce weights for SimpleWorld that satisfy the constraints.
# However, different algorithms cna provide different solutions.
# In this example multiple sets of weights satisfy the constraints, and it is not clear which is the 'best'.
# The basinhopping and differential evolution examples feel better because they make an attempt at global optimising.
# However they are stochastic, so you get different results if you don't set the seed.
# And, they are fiddly to paramterise compared to ipf and local minimisers.